<a href="https://colab.research.google.com/github/alagrin/masters_coursework/blob/master/RecidivismA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import tensorflow as tf
from tensorflow import keras
import math
import matplotlib.pyplot as plt

# The file names
RAW = 'compas-scores-raw.csv'
TWO_YEARS = 'compas-scores-two-years.csv'

# Date formats
# %m = zero-padded month, %d = zero-padded day, %y = two-digit year
RAW_DATE = '%m/%d/%y' 
TWO_YEARS_DATE = '%Y-%m-%d'

In [5]:
from google.colab import files
import os


def get_data(file_name):
  # Check if the file has already been uploaded
  exists = os.path.isfile(file_name)
  # If the file is not already uploaded, show the file upload widget. Notebooks
  # reset every 12 hours.
  if not exists:
    print("Select the file", file_name)
    uploaded = files.upload()
    
    for fn in uploaded.keys():
      print("Uploaded file {name} with length {length} bytes".format(
          name=fn, length=len(uploaded[fn])))
  else:
    print(file_name, "already uploaded.")
  # Save the contents of the file as a Pandas dataframe
  df = pd.read_csv(file_name)
  return df

df_raw = get_data(RAW)
df_all = get_data(TWO_YEARS)

compas-scores-raw.csv already uploaded.
Select the file compas-scores-two-years.csv


Saving compas-scores-two-years.csv to compas-scores-two-years.csv
Uploaded file compas-scores-two-years.csv with length 2546489 bytes


In [170]:
print("df_raw: Compas score raw")
display(df_raw.head(10))
print("df_all: Compas score two years")
display(df_all.head(5))

df_raw: Compas score raw


,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,ScaleSet_ID,ScaleSet,AssessmentReason,Language,LegalStatus,CustodyStatus,MaritalStatus,Screening_Date,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,pretrial,fisher,kevin,NaN,male,caucasian,1992-12-05,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,1,low,7,risk of violence,-2.08,4,low,new,1,0
1,50844,57167,51950,pretrial,fisher,kevin,NaN,male,caucasian,1992-12-05,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,1,low,8,risk of recidivism,-1.06,2,low,new,1,0
2,50844,57167,51950,pretrial,fisher,kevin,NaN,male,caucasian,1992-12-05,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,1,low,18,risk of failure to appear,15.00,1,low,new,1,0
3,50848,57174,51956,pretrial,kendall,kevin,NaN,male,caucasian,1984-09-16,22,risk and prescreen,intake,english,pretrial,jail inmate,married,2013-01-01,1,low,7,risk of violence,-2.84,2,low,new,1,0
4,50848,57174,51956,pretrial,kendall,kevin,NaN,male,caucasian,1984-09-16,22,risk and prescreen,intake,english,pretrial,jail inmate,married,2013-01-01,1,low,8,risk of recidivism,-1.50,1,low,new,1,0
5,50848,57174,51956,pretrial,kendall,kevin,NaN,male,caucasian,1984-09-16,22,risk and prescreen,intake,english,pretrial,jail inmate,married,2013-01-01,1,low,18,risk of failure to appear,19.00,3,low,new,1,0
6,50855,57181,51963,pretrial,dayes,daniel,NaN,male,african-american,1994-08-25,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,4,high,7,risk of violence,-1.20,8,high,new,1,0
7,50855,57181,51963,pretrial,dayes,daniel,NaN,male,african-american,1994-08-25,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,4,high,8,risk of recidivism,0.18,8,high,new,1,0
8,50855,57181,51963,pretrial,dayes,daniel,NaN,male,african-american,1994-08-25,22,risk and prescreen,intake,english,pretrial,jail inmate,single,2013-01-01,4,high,18,risk of failure to appear,13.00,1,low,new,1,0
9,50850,57176,51958,pretrial,debe,mikerlie,george,female,african-american,1994-10-09,22,risk and prescreen,intake,english,pretrial,jail inmate,significant other,2013-01-01,2,medium,7,risk of violence,-1.29,7,medium,new,1,0


df_all: Compas score two years


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,male,1947-04-18,69,greater than 45,other,0,1,0,0,0,-1.0,2013-08-13,2013-08-14,13011352cf10a,2013-08-13,NaT,1.0,f,aggravated assault w/firearm,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,1,low,2013-08-14,risk of violence,1,low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,male,1982-01-22,34,25 - 45,african-american,0,3,0,0,0,-1.0,2013-01-26,2013-02-05,13001275cf10a,2013-01-26,NaT,1.0,f,felony battery w/prior convict,1,13009779cf10a,(f3),NaN,2013-07-05,felony battery (dom strang),NaN,NaN,NaN,1,13009779cf10a,(f3),2013-07-05,felony battery (dom strang),risk of recidivism,3,low,2013-01-27,risk of violence,1,low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,male,1991-05-14,24,less than 25,african-american,0,4,0,1,4,-1.0,2013-04-13,2013-04-14,13005330cf10a,2013-04-13,NaT,1.0,f,possession of cocaine,1,13011511mm10a,(m1),0.0,2013-06-16,driving under the influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,4,low,2013-04-14,risk of violence,3,low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,male,1993-01-21,23,less than 25,african-american,0,8,1,0,1,NaN,NaT,NaT,13000570cf10a,2013-01-12,NaT,1.0,f,possession of cannabis,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,8,high,2013-01-13,risk of violence,6,medium,2013-01-13,NaT,NaT,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,male,1973-01-22,43,25 - 45,other,0,1,0,0,2,NaN,NaT,NaT,12014130cf10a,NaT,2013-01-09,76.0,f,arrest case no charge,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,1,low,2013-03-26,risk of violence,1,low,2013-03-26,NaT,NaT,2,0,1102,0,0


In [171]:
df_all.groupby("race")['id'].nunique() # check of demographics numbers

race
african-american    3696
asian                 32
caucasian           2454
hispanic             637
native american       18
other                377
Name: id, dtype: int64

In [182]:
'''Here we choose people on the record with their unique ID, race which factors into the evaluation, followed by info on whether or not
they were a repeat offender, their decile score corresponding to low/high in terms of number, and if they had repeated within the two years'''
df_test = df_all[["race", "age", "is_recid", "decile_score", "two_year_recid"]].copy()

df_all["race"] # this column identifies the race of the individual in question

df_all["age"] # age of the offender

df_all["is_recid"] # this column helps us when we need to evaluate whether the person was a repeat offender

df_all["decile_score"] # shows us the score whether the offender scored low or high on the scale of Compas scores

df_all["two_year_recid"] # shows if offender reoffended within a couple years of previous charge


0       0
1       1
2       1
3       0
4       0
       ..
7209    0
7210    0
7211    0
7212    0
7213    1
Name: two_year_recid, Length: 7214, dtype: int64

In [178]:
df_test.describe()

,age,is_recid,decile_score,two_year_recid
count,7214.000000,7214.000000,7214.000000,7214.000000
mean,34.817993,0.481148,4.509565,0.450652
std,11.888922,0.499679,2.856396,0.497593
min,18.000000,0.000000,1.000000,0.000000
25%,25.000000,0.000000,2.000000,0.000000
50%,31.000000,0.000000,4.000000,0.000000
75%,42.000000,1.000000,7.000000,1.000000
max,96.000000,1.000000,10.000000,1.000000


In [0]:
high_recid_black = df_test.query('two_year_recid == 1 and race == "African-American" and decile_score >= 5').count()

In [0]:
total_recid_black = df_test.query('race == "African-American" and decile_score >= 5').count()

In [14]:
# PPV rate confirmed ~0.63
rate = high_recid_black / total_recid_black
print(rate) 

race              0.629715
age               0.629715
is_recid          0.629715
decile_score      0.629715
two_year_recid    0.629715
dtype: float64


In [0]:
high_recid_white = df_test.query('two_year_recid == 1 and race == "Caucasian" and decile_score >= 5').count()

In [0]:
total_recid_white =  df_test.query('race == "Caucasian" and decile_score >= 5').count()

In [66]:
# PPV rate confirmed ~0.59
rate_white = high_recid_white / total_recid_white
print(rate_white)

id                0.591335
race              0.591335
is_recid          0.591335
decile_score      0.591335
two_year_recid    0.591335
dtype: float64

Claim 2 - false positive comparison



In [0]:
high_surv_black = df_test.query('two_year_recid == 0 and race == "African-American" and decile_score >= 5').count()

In [0]:
total_surv_black = df_test.query('race == "African-American" and two_year_recid == 0').count()

In [73]:
false_pos_black = high_surv_black/ total_surv_black
print(false_pos_black)

id                0.448468
race              0.448468
is_recid          0.448468
decile_score      0.448468
two_year_recid    0.448468
dtype: float64

In [0]:
high_surv_white = df_test.query('two_year_recid == 0 and race == "Caucasian" and decile_score >= 5').count()

In [0]:
total_surv_white = df_test.query('race == "Caucasian" and two_year_recid == 0').count()

In [76]:
false_pos_white = high_surv_white/ total_surv_white
print(false_pos_white)

id                0.234543
race              0.234543
is_recid          0.234543
decile_score      0.234543
two_year_recid    0.234543
dtype: float64

In [199]:
all_white_recid = df_test.query('race == "caucasian" and is_recid==1').count()
recid_white = df_test.query('race == "caucasian" and is_recid == 1 and decile_score <= 4').count()

false_neg_white = recid_white / all_white_recid
print("FNR for white: ", false_neg_white[0])

all_black_recid = df_test.query('race == "african-american" and is_recid == 1').count()
recid_black = df_test.query('race == "african-american" and is_recid == 1 and decile_score <= 4').count()

false_neg_black = recid_black / all_black_recid
print("FNR for black: ", false_neg_black[0])


FNR for white:  0.4897560975609756
FNR for black:  0.29027504911591356


# Part 2

In [0]:
# Rerun top 2 cells and then continue here

In [0]:
'''
RUN THIS CELL AS-IS
'''

# df = a dataframe,
# cols = a list of columns to convert
# d_format = the date format to use
def convert_col_to_date(df, cols, d_format):
  for col in cols:
    
    if df[col].dtype == np.object:
      # Drop any time info
      df[col] = df[col].str.replace(r'\s[\w*:]*','')
      df[col] = df.apply(lambda defendant:make_date(defendant, col, d_format), 
                         axis=1);
      
def make_date(defendant, col, d_format):
  try:
    return datetime.strptime(defendant[col], d_format)
  except:
    return None
    

# df = a dataframe,
# cols = a list of columns to convert
def convert_col_to_number(df, cols):
  for col in cols:
    if df[col].dtype == np.object:
      df[col] = pd.to_numeric(df[col])
      
# df = a dataframe
def convert_col_to_lower(df):
  cols = list(df)
  for col in cols:
    if df[col].dtype == np.object:
      df[col] = df[col].str.lower()

'''
CONVERT DATE COLUMNS
'''
raw_date_cols = ['DateOfBirth', 'Screening_Date']
two_year_date_cols = ['compas_screening_date', 'dob', 'c_jail_in', 'c_jail_out',
                'c_offense_date', 'c_arrest_date', 'r_offense_date', 
                'vr_offense_date', 'screening_date', 'v_screening_date',
                'in_custody', 'out_custody']
convert_col_to_date(df_raw, raw_date_cols, RAW_DATE)
convert_col_to_date(df_all, two_year_date_cols, TWO_YEARS_DATE)

# Important! - With two digit years, Python auto converts some years to the 21st
# century and others to the 20th. All dates of birth should be 20th century!
def make_20th_c(row, col):
  if row[col].to_pydatetime().date().year >= 2000:
      return row[col].to_pydatetime().date() - relativedelta(years=100)
  else:
    return row[col]
df_raw["DateOfBirth"] = df_raw.apply(lambda defendant:make_20th_c(defendant, "DateOfBirth"), axis=1)

'''
CONVERT NUMBER COLUMNS
'''
raw_number_cols = ['ScaleSet_ID', 'RecSupervisionLevel', 'Scale_ID', 'RawScore',
                  'DecileScore', 'IsCompleted', 'IsDeleted']
two_year_number_cols = ['age', 'juv_fel_count', 'decile_score', 'juv_misd_count',
                        'juv_other_count', 'priors_count', 
                        'days_b_screening_arrest', 'c_days_from_compas',
                       'is_recid', 'r_days_from_arrest', 'is_violent_recid', 
                       'decile_score', 'v_decile_score', 'priors_count', 'start', 
                        'end', 'event','two_year_recid']
convert_col_to_number(df_raw, raw_number_cols)
convert_col_to_number(df_all, two_year_number_cols)

'''
CONVERT ALL STRING COLUMNS TO LOWERCASE
'''
convert_col_to_lower(df_raw)
convert_col_to_lower(df_all)

In [160]:
df_all.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,male,1947-04-18,69,greater than 45,other,0,1,0,0,0,-1.0,2013-08-13,2013-08-14,13011352cf10a,2013-08-13,NaT,1.0,f,aggravated assault w/firearm,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,1,low,2013-08-14,risk of violence,1,low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,male,1982-01-22,34,25 - 45,african-american,0,3,0,0,0,-1.0,2013-01-26,2013-02-05,13001275cf10a,2013-01-26,NaT,1.0,f,felony battery w/prior convict,1,13009779cf10a,(f3),NaN,2013-07-05,felony battery (dom strang),NaN,NaN,NaN,1,13009779cf10a,(f3),2013-07-05,felony battery (dom strang),risk of recidivism,3,low,2013-01-27,risk of violence,1,low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,male,1991-05-14,24,less than 25,african-american,0,4,0,1,4,-1.0,2013-04-13,2013-04-14,13005330cf10a,2013-04-13,NaT,1.0,f,possession of cocaine,1,13011511mm10a,(m1),0.0,2013-06-16,driving under the influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,4,low,2013-04-14,risk of violence,3,low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,male,1993-01-21,23,less than 25,african-american,0,8,1,0,1,NaN,NaT,NaT,13000570cf10a,2013-01-12,NaT,1.0,f,possession of cannabis,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,8,high,2013-01-13,risk of violence,6,medium,2013-01-13,NaT,NaT,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,male,1973-01-22,43,25 - 45,other,0,1,0,0,2,NaN,NaT,NaT,12014130cf10a,NaT,2013-01-09,76.0,f,arrest case no charge,0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0,NaN,NaN,NaT,NaN,risk of recidivism,1,low,2013-03-26,risk of violence,1,low,2013-03-26,NaT,NaT,2,0,1102,0,0


In [162]:
df_basic = df_all[["race", "sex", "age", "priors_count", "is_recid"]].copy()
'''We start by trying to focus on key features including race, sex, age, prior crime count, and a check on recidivism
The focus for the other models will be to equalize sample sizes and to remove race from the equation to see what's possible'''

"We start by trying to focus on key features including race, sex, age, prior crime count, and a check on recidivism\nThe focus for the other models will be to equalize sample sizes and to remove race from the equation to see what's possible"

Preprocessing Data for running through model by subgroup

In [0]:
def convert_race(row):
  if row["race"] == "african-american":
    return 0
  elif row["race"] == "caucasian":
    return 1
  else:
    return 2

def convert_sex(row):
  if row["sex"] == "male":
    return 0
  elif row["sex"] == "female":
    return 1
  else:
    return 2
  
def normalize_age(row, max_age):
  return row["age"] / max_age


# The preferred way to normalize categorical data:
df_basic = pd.get_dummies(df_basic,columns=["race","sex"], prefix=["race","sex"])

df_basic["age"] = df_basic.apply(lambda defendant: normalize_age(defendant, df_basic["age"].max()), axis=1)

# Further preprocessing to scale priors count
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = df_basic['priors_count'].values
df_basic['priors_count'] = scaler.fit_transform(x.reshape(-1,1))

In [0]:
# Split into train, validation, test

# Total amount of data
data_count = len(df_basic.index)

# Split into train - test (80-20)
train_total = math.floor(data_count * 0.8)
test_count = data_count - train_total

# Split the training set into train and validation (80-20)
train_count = math.floor(train_total * 0.8)
validation_count = train_total - train_count

# Create the sets - assignment should really be randomized!
df_train = df_basic[:train_count]
df_validation = df_basic[train_count:train_count+validation_count]
df_test = df_basic[train_count+validation_count:]

In [0]:
# Separate the labels (is_recid) from the input data

# x_ represents feature vectors, y_ represents the associated class labels

features = df_train.columns.tolist()
features.remove("is_recid")

x_train = df_train[features].copy()
y_train = df_train["is_recid"].copy()

x_validation = df_validation[features].copy()
y_validation = df_validation["is_recid"].copy()

x_test = df_test[features].copy()
y_test = df_test["is_recid"].copy()

In [0]:
# Building a classifier to predict recidivism using only race, gender, and age
NUM_FEATURES = len(features)
NUM_NODES = 32
OUTPUT = 1

# Initialising the classifier
classifier = keras.Sequential()

# Adding the input layer and the first hidden layer
# input_dim is the number of features you will input into the classifier
classifier.add(keras.layers.Dense(units=NUM_NODES, kernel_initializer='uniform', 
                                  activation='relu', input_dim=NUM_FEATURES))

# Adding a second hidden layer
classifier.add(keras.layers.Dense(units=NUM_NODES, kernel_initializer='uniform', 
                                  activation='relu'))

# Adding a second hidden layer
classifier.add(keras.layers.Dense(units=NUM_NODES, kernel_initializer='uniform', 
                                  activation='relu'))


# Add the output layer
classifier.add(keras.layers.Dense(units=OUTPUT, activation=tf.nn.sigmoid))

# Compile the model
classifier.compile(optimizer=tf.train.AdamOptimizer(), loss='binary_crossentropy',
              metrics=['accuracy'])

In [167]:
# Train the model
# So far, we've been working with DataFrames (pandas)
# Sequential.fit needs the data in array format. To convert, use df.values e.g.
# x_train.values

history = classifier.fit(x_train.values,
                    y_train.values,
                    epochs=60,
                    batch_size=512,
                    validation_data=(x_validation.values, y_validation.values),
                    verbose=1)

Train on 4616 samples, validate on 1155 samples
Epoch 1/60
4616/4616 [==============================] - 0s 58us/sample - loss: 0.6927 - acc: 0.5212 - val_loss: 0.6929 - val_acc: 0.5039
Epoch 2/60
4616/4616 [==============================] - 0s 8us/sample - loss: 0.6915 - acc: 0.5249 - val_loss: 0.6923 - val_acc: 0.5039
Epoch 3/60
4616/4616 [==============================] - 0s 9us/sample - loss: 0.6900 - acc: 0.5249 - val_loss: 0.6908 - val_acc: 0.5039
Epoch 4/60
4616/4616 [==============================] - 0s 9us/sample - loss: 0.6875 - acc: 0.5249 - val_loss: 0.6891 - val_acc: 0.5039
Epoch 5/60
4616/4616 [==============================] - 0s 10us/sample - loss: 0.6847 - acc: 0.5249 - val_loss: 0.6860 - val_acc: 0.5039
Epoch 6/60
4616/4616 [==============================] - 0s 10us/sample - loss: 0.6805 - acc: 0.5375 - val_loss: 0.6820 - val_acc: 0.5680
Epoch 7/60
4616/4616 [==============================] - 0s 8us/sample - loss: 0.6757 - acc: 0.6070 - val_loss: 0.6789 - val_acc: 0.61

In [168]:
# Evaluation = current numbers seem off and not sure what's going on...
results = classifier.evaluate(x_test, y_test)

print(classifier.metrics_names)
print(results)

1443/1443 [==============================] - 0s 46us/sample - loss: 0.6152 - acc: 0.6708
['loss', 'acc']
[0.6151736778925462, 0.67082465]


In [0]:
# Trying to separate the data w/ subset running white and black so we can then see if model gives FPR, FN, etc at more equal rate

race_cauc = df_test.loc[df_test["race_caucasian"] == 1]
race_afam = df_test.loc[df_test["race_african-american"] == 1]

x_cauc_test = race_cauc[features].copy()
y_cauc_test = race_cauc['is_recid'].copy()

x_afam_test = race_afam[features].copy()
y_afam_test = race_afam['is_recid'].copy()

In [0]:
# x_cauc_test.head()

In [0]:
# Now need to break down by demographics and compare the rates between groups

def compute_eval_metrics(references, predictions):
  tn, fp, fn, tp = confusion_matrix(references, predictions).ravel()
  precision = tp / float(tp + fp)
  recall = tp / float(tp + fn)
  false_positive_rate = fp / float(fp + tn)
  false_negative_rate = fn / float(fn + tp)
  return precision, recall, false_positive_rate, false_negative_rate

In [0]:
# Running the isolated subgroups through the model and seeing what predictions we get
y_pred_cauc = classifier.predict_classes(x_cauc_test)
y_pred_afam = classifier.predict_classes(x_afam_test)

In [21]:
# Expecting to run metrics and see for each subgroup y_test_cauc and y_test_afam a better or more similar FPR so that we can achevie 
# something close to classification parity

from sklearn.metrics import confusion_matrix

ordinal_overall_metrics = compute_eval_metrics(y_cauc_test, y_pred_cauc)
print("Precision = ", ordinal_overall_metrics[0], "Recall = ", ordinal_overall_metrics[1], "Overall FPR =", ordinal_overall_metrics[2], "Overall FNR =", ordinal_overall_metrics[3])

(0.5824742268041238, 0.5885416666666666, 0.2852112676056338, 0.4114583333333333)
Precision =  0.5824742268041238 Recall =  0.5885416666666666 Overall FPR = 0.2852112676056338 Overall FNR = 0.4114583333333333


In [0]:
# New feature selection, remove race and just use age, violent recid, age_cat, priors_count, charge degree?, two year recid as well...

In [0]:
ordinal_overall_metrics = compute_eval_metrics(y_afam_test, y_pred_afam)
print("Precision = ", ordinal_overall_metrics[0], "Recall = ", ordinal_overall_metrics[1], "Overall FPR =", ordinal_overall_metrics[2], "Overall FNR =", ordinal_overall_metrics[3])

In [0]:
# Make sure main model is run once and then rerun the build of classifier above

In [0]:
df_no_race = df_all[["sex", "age", "priors_count", "is_recid"]].copy()

In [0]:
def convert_sex(row):
  if row["sex"] == "male":
    return 0
  elif row["sex"] == "female":
    return 1
  else:
    return 2
  
def normalize_age(row, max_age):
  return row["age"] / max_age


# The preferred way to normalize categorical data:
df_2 = pd.get_dummies(df_no_race,columns=["sex"], prefix=["sex"])

df_2["age"] = df_2.apply(lambda defendant: normalize_age(defendant, df_2["age"].max()), axis=1)

# Further preprocessing to scale priors count
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = df_2['priors_count'].values
df_2['priors_count'] = scaler.fit_transform(x.reshape(-1,1))

In [0]:
features = df_2.columns.tolist()
features.remove("is_recid")

x_train = df_train[features].copy()
y_train = df_train["is_recid"].copy()

x_validation = df_validation[features].copy()
y_validation = df_validation["is_recid"].copy()

x_test = df_test[features].copy()
y_test = df_test["is_recid"].copy()

In [0]:
print(features)

In [0]:
# df_test.head()
race_cauc = df_test.loc[df_test["race_Caucasian"] == 1]
race_afam = df_test.loc[df_test["race_African-American"] == 1]

x_cauc_test = race_cauc[features].copy()
y_cauc_test = race_cauc['is_recid'].copy()

x_afam_test = race_afam[features].copy()
y_afam_test = race_afam['is_recid'].copy()

In [0]:
# Running the isolated subgroups through the model and seeing what predictions we get
y_pred_cauc = classifier.predict_classes(x_cauc_test)
y_pred_afam = classifier.predict_classes(x_afam_test)

In [79]:
from sklearn.metrics import confusion_matrix

ordinal_overall_metrics = compute_eval_metrics(y_cauc_test, y_pred_cauc)
print("Precision = ", ordinal_overall_metrics[0], "Recall = ", ordinal_overall_metrics[1], "Overall FPR =", ordinal_overall_metrics[2], "Overall FNR =", ordinal_overall_metrics[3])

Precision =  0.46808510638297873 Recall =  0.34375 Overall FPR = 0.2640845070422535 Overall FNR = 0.65625


In [80]:
ordinal_overall_metrics = compute_eval_metrics(y_afam_test, y_pred_afam)
print("Precision = ", ordinal_overall_metrics[0], "Recall = ", ordinal_overall_metrics[1], "Overall FPR =", ordinal_overall_metrics[2], "Overall FNR =", ordinal_overall_metrics[3])

Precision =  0.6479400749063671 Recall =  0.4128878281622912 Overall FPR = 0.28923076923076924 Overall FNR = 0.5871121718377088


Inadvertently here we are seeing a very close false positive rate. As we can see above running predictions on columns maintained of age, sex, and prior counts/arrests. This is as close as I was able to acheive in terms of similarity of FPR's between the two racial groups. The goal overall being to eliminate bias and improve prediction fairness

In [97]:
'''Only other method that remains is to equalize the number of samples of Caucasian and African-Americans in the sample we are
building the model off of.'''

'Only other method that remains is to equalize the number of samples of Caucasian and African-Americans in the sample we are\nbuilding the model off of.'

In [126]:
# df_all.query('race == "Caucasian"').count() and then get equal sample size by using 2454
df_similar = df_all.query('race == "Caucasian" or race == "African-American"').sample(frac=0.5)
df_similar['race'].describe()

count                 3075
unique                   2
top       African-American
freq                  1855
Name: race, dtype: object

In [0]:
def convert_race(row):
  if row["race"] == "african-american":
    return 0
  elif row["race"] == "caucasian":
    return 1
  else:
    return 2

def convert_sex(row):
  if row["sex"] == "male":
    return 0
  elif row["sex"] == "female":
    return 1
  else:
    return 2
  
def normalize_age(row, max_age):
  return row["age"] / max_age


# The preferred way to normalize categorical data:
df_new = pd.get_dummies(df_similar,columns=["race","sex"], prefix=["race","sex"])

df_new["age"] = df_new.apply(lambda defendant: normalize_age(defendant, df_new["age"].max()), axis=1)

# Further preprocessing to scale priors count
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = df_new['priors_count'].values
df_new['priors_count'] = scaler.fit_transform(x.reshape(-1,1))

In [0]:
# Split into train, validation, test

# Total amount of data
data_count = len(df_similar.index)

# Split into train - test (80-20)
train_total = math.floor(data_count * 0.8)
test_count = data_count - train_total

# Split the training set into train and validation (80-20)
train_count = math.floor(train_total * 0.8)
validation_count = train_total - train_count

# Create the sets - assignment should really be randomized!
df_train = df_similar[:train_count]
df_validation = df_similar[train_count:train_count+validation_count]
df_test = df_similar[train_count+validation_count:]

In [0]:
features = df_similar.columns.tolist()
features.remove("is_recid")

x_train = df_train[features].copy()
y_train = df_train["is_recid"].copy()

x_validation = df_validation[features].copy()
y_validation = df_validation["is_recid"].copy()

x_test = df_test[features].copy()
y_test = df_test["is_recid"].copy()

In [130]:
print(features)

['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'r_case_number', 'r_charge_degree', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'type_of_assessment', 'decile_score.1', 'score_text', 'screening_date', 'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1', 'start', 'end', 'event', 'two_year_recid']


In [0]:
'''Attempted but could not figure out a way to get equal sample sizes for both groups to run through but ideally would see that a more equal
sample size of the two races would bring the FPR and other metrics closer to each other - more fair since we wouldn't have inherently
skewed data'''